In [4]:
from pyspark.sql import SparkSession

# Создать Spark сессию
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("TestApp") \
    .getOrCreate()

# Создать тестовую таблицу
data = [
    (1, "Alice", 25),
    (2, "Bob", 30),
    (3, "Charlie", 35)
]

df = spark.createDataFrame(data, ["id", "name", "age"])

# Показать данные
df.show()

# Записать в HDFS (Parquet формат)
df.coalesce(1).write.mode("overwrite").parquet("hdfs://namenode:9000/user/hive/warehouse/test_table")
# df.write.mode("overwrite").parquet("hdfs://namenode:9000/user/hive/warehouse/test_table")

print("✓ Таблица записана в HDFS: hdfs://namenode:9000/user/hive/warehouse/test_table")

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 25|
|  2|    Bob| 30|
|  3|Charlie| 35|
+---+-------+---+



[Stage 2:>                                                          (0 + 1) / 1]

✓ Таблица записана в HDFS: hdfs://namenode:9000/user/hive/warehouse/test_table


In [2]:
# Прочитать обратно из HDFS
df_read = spark.read.parquet("hdfs://namenode:9000/user/hive/warehouse/test_table")
df_read.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  2|    Bob| 30|
|  3|Charlie| 35|
|  1|  Alice| 25|
+---+-------+---+



In [2]:
spark.stop()

In [1]:
from pyspark.sql import SparkSession

# Создать Spark сессию с поддержкой Hive
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("CreateHiveTableApp") \
    .config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

# Создать тестовые данные
data = [
    (1, "Alice", 25),
    (2, "Bob", 30),
    (3, "Charlie", 35)
]

df = spark.createDataFrame(data, ["id", "name", "age"])

# Показать данные
print("Исходные данные:")
df.show()

# Имя базы данных и таблицы
db_name = "default"
table_name = "test_people_table"

# Записать как управляемую (managed) таблицу Hive
# saveAsTable автоматически создает таблицу в Hive Metastore и кладет данные в spark.sql.warehouse.dir (или путь по умолчанию в HDFS)
df.write \
    .mode("overwrite") \
    .saveAsTable(f"{db_name}.{table_name}")

print(f"✓ Таблица {db_name}.{table_name} успешно создана.")

# Проверка: читаем из таблицы через SQL
print("Чтение из созданной таблицы через Spark SQL:")
spark.sql(f"SELECT * FROM {db_name}.{table_name}").show()

# spark.stop()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/18 19:57:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Исходные данные:


+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 25|
|  2|    Bob| 30|
|  3|Charlie| 35|
+---+-------+---+



26/01/18 19:57:50 WARN metastore: Failed to connect to the MetaStore Server...
26/01/18 19:57:51 WARN metastore: Failed to connect to the MetaStore Server...
26/01/18 19:57:52 WARN metastore: Failed to connect to the MetaStore Server...
26/01/18 19:57:53 WARN HiveClientImpl: HiveClient got thrift exception, destroying client and retrying (0 tries remaining)
org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
	at org.apache.hadoop.hive.ql.metadata.Hive.getDatabase(Hive.java:1567)
	at org.apache.hadoop.hive.ql.metadata.Hive.databaseExists(Hive.java:1552)
	at org.apache.spark.sql.hive.client.Shim_v0_12.databaseExists(HiveShim.scala:609)
	at org.apache.spark.sql.hive.client.HiveClientImpl.$anonfun$databaseExists$1(HiveClientImpl.scala:406)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at org.apache.spark.sql.hive.client.HiveClientImpl.$anonfun$withHive

AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient